In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
%matplotlib inline

dataset = pd.read_csv('part-00000-14adf10f-5ab2-4d08-b8c3-8efab23e7202-c000.csv')
dataset.head()


,cpu_pct,timestamp,mem_pct
0,0.345414,1564112951102,0.410128
1,0.345414,1564112951102,0.410128
2,0.345414,1564112951102,0.410128
3,0.345414,1564112951102,0.410128
4,0.345414,1564112951102,0.410128


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler

data = dataset.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,1000),columns=['mem_pct', 'timestamp'])

data.head()
for i in range(0, 1000):
    new_data['timestamp'][i]= data['timestamp'][i]
    new_data['mem_pct'][i] = data['mem_pct'][i]

new_data.index = new_data.timestamp
new_data.drop('timestamp', axis=1, inplace=True)

dataset = new_data.values

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [4]:
train,valid = train_test_split(new_data, test_size=0.2)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=10, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=10))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
mempct = model.predict(X_test)
mempct = scaler.inverse_transform(mempct)

rms=np.sqrt(np.mean(np.power((valid-mempct),2)))
rms

Epoch 1/1
 - 68s - loss: 0.0000e+00


mem_pct    2.110752e-09
dtype: float64

In [ ]:

#for plotting
train = new_data[:200]
valid = new_data[200:]
valid['Predictions'] = mempct
plt.plot(train['mem_pct'])
plt.plot(valid[['mem_pct','Predictions']])